In [277]:
import pandas as pd
import numpy as np
import tejapi
import statsmodels.formula.api as smf
import datetime as dt
from dateutil.relativedelta import *
tejapi.ApiConfig.api_key = "maQNDP8Vb5bBB8b55kCkR2TC6s65Eg"
tejapi.ApiConfig.api_base="http://10.10.10.66"
tejapi.ApiConfig.ignoretz = True

In [144]:
df1 = tejapi.get('TWN/AIND', #從TEJ api撈取所需要的資料
                chinese_column_name = True,
                paginate = True,
#                 mdate = {'gt':'2018-01-01'},
                opts={'columns':['coid','mdate', 'dflt_d','fail_fg']})

In [320]:
df1['危機發生日'].sort_values(ascending=False)

None
1939   2022-08-08
2953   2022-08-05
2399   2022-08-03
1382   2022-07-28
745    2022-07-26
          ...    
3106          NaT
3107          NaT
3108          NaT
3109          NaT
3110          NaT
Name: 危機發生日, Length: 3111, dtype: datetime64[ns]

In [323]:
df1.loc[1939]

公司簡稱                     6103
目前狀態      2022-01-01 00:00:00
危機發生日     2022-08-08 00:00:00
危機事件類別                      E
Name: 1939, dtype: object

In [170]:
df2 = df1[df1['危機事件類別'] != ''] #把有發生違約事件的篩選出來

In [175]:
df2['年/月'] = df2[['危機發生日']].applymap(lambda x: x.strftime('%Y-%m')).astype('datetime64') #把日期都換成月初

/var/folders/gn/7b_mmd0s64q0kkr42rncl0jc0000gn/T/ipykernel_26068/2932515843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['年/月'] = df2[['危機發生日']].applymap(lambda x: x.strftime('%Y-%m')).astype('datetime64')


In [177]:
df2['月'] = df2['年/月'].dt.month #取月份出來

/var/folders/gn/7b_mmd0s64q0kkr42rncl0jc0000gn/T/ipykernel_26068/1595404685.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['月'] = df2['年/月'].dt.month


In [193]:
df2.reset_index(inplace = True)

In [202]:
for i in range(len(df2.index)): #為了之後對上財報資料，先將日期處理好
    if df2['月'][i] == 1 or df2['月'][i] == 4 or df2['月'][i] == 7 or df2['月'][i] == 10:
        df2['年/月'][i] = df2['年/月'][i]+ relativedelta(months = +2)
    if df2['月'][i] == 2 or df2['月'][i] == 5 or df2['月'][i] == 8 or df2['月'][i] == 11:
        df2['年/月'][i] = df2['年/月'][i]+ relativedelta(months = +1)

/var/folders/gn/7b_mmd0s64q0kkr42rncl0jc0000gn/T/ipykernel_26068/3712162270.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['年/月'][i] = df2['年/月'][i]+ relativedelta(months = +2)
/var/folders/gn/7b_mmd0s64q0kkr42rncl0jc0000gn/T/ipykernel_26068/3712162270.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['年/月'][i] = df2['年/月'][i]+ relativedelta(months = +1)


In [207]:
df2['Y'] = 1 #把所有危機事件類別設成1

/var/folders/gn/7b_mmd0s64q0kkr42rncl0jc0000gn/T/ipykernel_26068/2072358197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Y'] = 1


In [209]:
df2.rename(columns= {'公司簡稱':'公司'}, inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


抓變數會用到的財報資料

In [12]:
a1 = tejapi.get('TWN/AIFIN', #從TEJ api撈取所需要的資料
                chinese_column_name = True,
                paginate = True,
                mdate = {'gt':'2008-01-01', 'lt':'2011-01-01'},
                acc_code = ['R678', '0010','2341','2402','MV','1000', 'R607','R11V', 'R505'])

In [14]:
a2 = tejapi.get('TWN/AIFIN', #從TEJ api撈取所需要的資料
                chinese_column_name = True,
                paginate = True,
                mdate = {'gt':'2011-01-01', 'lt':'2014-01-01'},
                acc_code = ['R678', '0010','2341','2402','MV','1000', 'R607','R11V', 'R505'])

In [25]:
a3 = tejapi.get('TWN/AIFIN', #從TEJ api撈取所需要的資料
                chinese_column_name = True,
                paginate = True,
                mdate = {'gt':'2014-01-01', 'lt':'2017-01-01'},
                acc_code = ['R678', '0010','2341','2402','MV','1000', 'R607','R11V', 'R505'])

In [26]:
a4 = tejapi.get('TWN/AIFIN', #從TEJ api撈取所需要的資料
                chinese_column_name = True,
                paginate = True,
                mdate = {'gt':'2017-01-01', 'lt':'2020-01-01'},
                acc_code = ['R678', '0010','2341','2402','MV','1000', 'R607','R11V', 'R505'])

In [27]:
a5 = tejapi.get('TWN/AIFIN', #從TEJ api撈取所需要的資料
                chinese_column_name = True,
                paginate = True,
                mdate = {'gt':'2020-01-01'},
                acc_code = ['R678', '0010','2341','2402','MV','1000', 'R607','R11V', 'R505'])

In [213]:
acc = pd.concat([a1,a2,a3,a4,a5])

In [214]:
acc1 = acc.pivot_table(values='數值', index=['公司','年/月'], columns='會計科目') #用pivot table將會計科目放到columns上

In [215]:
acc1

會計科目                   0010      1000      2341     2402  MV  R11V   R505  \
公司     年/月                                                                  
000016 2008-06-01  853115.0  206761.0   96354.0  38116.0 NaN   NaN  24.24   
       2008-12-01  794817.0  136689.0  108128.0  18324.0 NaN   NaN  17.20   
       2009-06-01  789105.0  127192.0  113626.0  30467.0 NaN   NaN  16.12   
       2009-12-01  828313.0  158918.0  121108.0  11891.0 NaN   NaN  19.19   
       2010-06-01  723437.0  293459.0  129978.0  28978.0 NaN   NaN  40.56   
...                     ...       ...       ...      ...  ..   ...    ...   
Z5389  2021-03-01  671435.0  506622.0  -69187.0      NaN NaN  3.12  75.45   
       2021-06-01  666327.0  504462.0  -72135.0      NaN NaN  2.66  75.71   
       2021-09-01  741529.0  477884.0  -78355.0      NaN NaN  1.59  64.45   
       2021-12-01  669558.0  413791.0  -86233.0      NaN NaN  0.44  61.80   
       2022-03-01  791483.0  503928.0  -54445.0      NaN NaN  4.35  63.67   

會計科目               R607  R678  
公司     年/月                     
000016 2008-06-01  0.67   NaN  
       2008-12-01  0.63   NaN  
       2009-06-01  0.50   NaN  
       2009-12-01  0.52   NaN  
       2010-06-01  0.71   NaN  
...                 ...   ...  
Z5389  2021-03-01  0.38   NaN  
       2021-06-01  0.29   NaN  
       2021-09-01  0.24   NaN  
       2021-12-01  0.24   NaN  
       2022-03-01  0.46   NaN  

[125489 rows x 9 columns]

In [216]:
acc1['X1'] = (acc1['R678']/acc1['0010'])*100
acc1['X2'] = (acc1['2341']/acc1['0010'])*100
acc1['X3'] = (acc1['2402']/acc1['0010'])*100
acc1['X4'] = (acc1['MV']/acc1['1000'])*100
acc1 = acc1.rename(columns = {'R607':'X5', 'R11V':'X6', 'R505':'X7'})
acc2 = acc1[['X1','X2','X3','X4','X5','X6','X7']]

In [228]:
acc2.reset_index(inplace=True)

In [231]:
df3 = pd.merge(acc2, df2[['公司','年/月','Y']], how='outer')

In [233]:
df3['Y'] = df3['Y'].replace(np.nan, 0) #把沒有發生危機的設為0

In [237]:
df3 = df3.dropna()

In [253]:
df3['X4'] = df3['X4'].drop([59690,59688]) #把無限大的值去掉

/var/folders/gn/7b_mmd0s64q0kkr42rncl0jc0000gn/T/ipykernel_26068/227073911.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['X4'] = df3['X4'].drop([59690,59688]) #把無限大的值去掉


linear probility model

In [254]:
result_ols = smf.ols('Y ~ X1 + X2 + X3 + X4 + X5 + X6 + X7', data=df3).fit()
print(result_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     19.83
Date:                Wed, 10 Aug 2022   Prob (F-statistic):           9.62e-27
Time:                        10:54:15   Log-Likelihood:                 76114.
No. Observations:               74040   AIC:                        -1.522e+05
Df Residuals:                   74032   BIC:                        -1.521e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0031      0.001      2.217      0.0

probit model

In [255]:
result_probit = smf.probit('Y ~ X1 + X2 + X3 + X4 + X5 + X6 + X7 ', data=df3).fit()
print(result_probit.summary())

Optimization terminated successfully.
         Current function value: 0.043805
         Iterations 8
                          Probit Regression Results                           
Dep. Variable:                      Y   No. Observations:                74040
Model:                         Probit   Df Residuals:                    74032
Method:                           MLE   Df Model:                            7
Date:                Wed, 10 Aug 2022   Pseudo R-squ.:                 0.01514
Time:                        10:54:18   Log-Likelihood:                -3243.3
converged:                       True   LL-Null:                       -3293.2
Covariance Type:            nonrobust   LLR p-value:                 1.229e-18
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.6246      0.068    -38.690      0.000      -2.758      -2.492
X1             0.0004      0.

logit model

In [256]:
result_Logit = smf.logit('Y ~ X1 + X2 + X3 + X4 + X5 + X6 + X7  ', data=df3).fit()
print(result_Logit.summary())

Optimization terminated successfully.
         Current function value: 0.043828
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                74040
Model:                          Logit   Df Residuals:                    74032
Method:                           MLE   Df Model:                            7
Date:                Wed, 10 Aug 2022   Pseudo R-squ.:                 0.01461
Time:                        10:54:19   Log-Likelihood:                -3245.0
converged:                       True   LL-Null:                       -3293.2
Covariance Type:            nonrobust   LLR p-value:                 6.405e-18
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -5.3641      0.183    -29.274      0.000      -5.723      -5.005
X1             0.0010      0.

台積電

In [257]:
TSMC = df3[df3['公司']=='2330']

In [258]:
a = TSMC.iloc[-1:,:] #取得最新一筆資料

In [286]:
a

,公司,年/月,X1,X2,X3,X4,X5,X6,X7,Y
22741,2330,2022-03-01,22.525489,51.050002,5.734811,926.351819,0.51,5.66,41.86,0.0


In [259]:
result_ols.predict(a) #linear probility model

22741    0.008783
dtype: float64

In [260]:
result_probit.predict(a) #probit model

22741    0.008256
dtype: float64

In [261]:
result_Logit.predict(a) #logit model

22741    0.008292
dtype: float64

英瑞ky

In [262]:
Enterex = df3[df3['公司']=='1592']

In [263]:
b = Enterex.iloc[-1:,:]

In [264]:
result_ols.predict(b) #linear probility model

12385    0.015805
dtype: float64

In [265]:
result_probit.predict(b) #probit model

12385    0.017519
dtype: float64

In [266]:
result_Logit.predict(b) #logit model

12385    0.016884
dtype: float64

華映

In [267]:
CPT = df3[df3['公司']=='2475']

In [335]:
d = CPT.iloc[-1:,:]

In [269]:
result_ols.predict(d) #linear probility model

28962    0.031108
dtype: float64

In [270]:
result_probit.predict(d) #probit model

28962    0.030952
dtype: float64

In [271]:
result_Logit.predict(d) #logit model

28962    0.028548
dtype: float64

聯發科

In [336]:
mediatek = df3[df3['公司']=='2454']

In [337]:
e = mediatek.iloc[-1:,:]

In [338]:
result_ols.predict(e) #linear probility model

27973    0.005946
dtype: float64

In [339]:
result_probit.predict(e) #probit model

27973    0.006113
dtype: float64

In [340]:
result_Logit.predict(e) #logit model

27973    0.006254
dtype: float64